In [ ]:
# Web scraping da Agência SEBRAE de Notícias

from bs4 import BeautifulSoup
import requests
import pandas as pd

# Dicionário de estados e suas respectivas URLs
urls_estados = {
    "BR": "https://agenciasebrae.com.br/ultimas-noticias/page/",
    "AC": "https://ac.agenciasebrae.com.br/ultimas-noticias/page/",
    "AL": "https://al.agenciasebrae.com.br/ultimas-noticias/page/",
    "AM": "https://am.agenciasebrae.com.br/ultimas-noticias/page/",
    "AP": "https://ap.agenciasebrae.com.br/ultimas-noticias/page/",
    "BA": "https://ba.agenciasebrae.com.br/ultimas-noticias/page/",
    "CE": "https://ce.agenciasebrae.com.br/ultimas-noticias/page/",
    "DF": "https://df.agenciasebrae.com.br/ultimas-noticias/page/",
    "ES": "https://es.agenciasebrae.com.br/ultimas-noticias/page/",
    "GO": "https://go.agenciasebrae.com.br/ultimas-noticias/page/",
    "MA": "https://ma.agenciasebrae.com.br/ultimas-noticias/page/",
    "MG": "https://mg.agenciasebrae.com.br/ultimas-noticias/page/",
    "MS": "https://ms.agenciasebrae.com.br/ultimas-noticias/page/",
    "MT": "https://mt.agenciasebrae.com.br/ultimas-noticias/page/",
    "PA": "https://pa.agenciasebrae.com.br/ultimas-noticias/page/",
    "PB": "https://pb.agenciasebrae.com.br/ultimas-noticias/page/",
    "PE": "https://pe.agenciasebrae.com.br/ultimas-noticias/page/",
    "PI": "https://pi.agenciasebrae.com.br/ultimas-noticias/page/",
    "PR": "https://pr.agenciasebrae.com.br/ultimas-noticias/page/",
    "RJ": "https://rj.agenciasebrae.com.br/ultimas-noticias/page/",
    "RN": "https://rn.agenciasebrae.com.br/ultimas-noticias/page/",
    "RO": "https://ro.agenciasebrae.com.br/ultimas-noticias/page/",
    "RR": "https://rr.agenciasebrae.com.br/ultimas-noticias/page/",
    "RS": "https://rs.agenciasebrae.com.br/ultimas-noticias/page/",
    "SC": "https://sc.agenciasebrae.com.br/ultimas-noticias/page/",
    "SE": "https://se.agenciasebrae.com.br/ultimas-noticias/page/",
    "SP": "https://sp.agenciasebrae.com.br/ultimas-noticias/page/",
    "TO": "https://to.agenciasebrae.com.br/ultimas-noticias/page/"
}

# Listas para armazenar os dados extraídos
estados = []  # Armazenar o estado onde foi publicada cada notícia
links = []  # Armazenar o link de cada notícia
datas = []  # Armazenar a data de publicação de cada notícia
editoriais = []  # Armazenar o editorial de cada notícia
titulos = []  # Armazenar o título de cada notícia
subtitulos = []  # Armazenar o subtítulo de cada notícia
textos = []  # Armazenar o texto de cada notícia

# Loop para web scraping da página de notícias de cada estado
for estado, base_url in urls_estados.items():
    # Iniciar a contagem de páginas a partir da primeira
    pagina_inicial = 1
    
    # Criar um loop contínuo até ser interrompido
    while True:
        url = base_url + str(pagina_inicial) + "/"
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Verificar se a página de erro foi carregada (interrupção)
        error_message = soup.find('div', class_='info')
        if error_message and 'Não encontramos essa página' in error_message.text:
            print(f'Página de erro encontrada em {url}. Parando a busca.')
            break

        # Extrair estado e link de cada notícia
        for h3 in soup.find_all('h3', class_='title'):
            estados.append(estado)  # Adiciona o estado correspondente
            links.append(h3.find('a')['href'])  # Adiciona o link correspondente
        
        # Incrementar o número da página para a próxima iteração
        pagina_inicial += 1

# Extrair data de publicação, editorial, título, subtítulo e texto de cada notícia
for link in links:
    # Tratar erro de redirecionamento excessivo
    try:
        response_noticia = requests.get(link)
    except requests.TooManyRedirects:
        print(f"Erro de redirecionamento excessivo para {link}.")
        error_message = "Erro de redirecionamento excessivo"
        datas.append(error_message)
        editoriais.append(error_message)
        titulos.append(error_message)
        subtitulos.append(error_message)
        textos.append(error_message)
        continue

    soup_noticia = BeautifulSoup(response_noticia.text, 'html.parser')

    # Extrair a data de publicação
    time_tag = soup_noticia.find('time', {'data-datetime': True})
    if time_tag:
        datas.append(time_tag['data-datetime'])
    else:
        datas.append("Data não disponível")

    # Extrair o editorial
    editorial_tag = soup_noticia.find('div', class_='editoria-crumb')
    if editorial_tag:
        editoriais.append(editorial_tag.get_text(strip=True))
    else:
        editoriais.append("Editorial não disponível")

    # Extrair o título
    titulo_tag = soup_noticia.find('h2')
    if titulo_tag:
        titulos.append(titulo_tag.text.strip())
    else:
        titulos.append("Título não disponível")

    # Extrair o subtítulo
    subtitulo_tag = soup_noticia.find('em', class_='sub-title')
    if subtitulo_tag:
        subtitulos.append(subtitulo_tag.text.strip())
    else:
        subtitulos.append("Subtítulo não disponível")

    # Extrair o texto
    paragraphs = [p.text for p in soup_noticia.select('.text-content p')]
    textos.append('\n'.join(paragraphs))

# Extrair tamanho de cada lista (teste)
print("Tamanho de cada lista: {}, {}, {}, {}, {}, {}, {}".format(len(estados), len(links), len(datas), len(editoriais), len(titulos), len(subtitulos), len(textos)))

# Criar DataFrame a partir das listas
df = pd.DataFrame({
    'Estado': estados,
    'Link': links,
    'Data de Publicação': datas,
    'Editorial': editoriais,
    'Título': titulos,
    'Subtítulo': subtitulos,
    'Texto': textos
})

# Salvar DataFrame em um arquivo
df.to_excel('ws_agencia_sebrae_noticias.xlsx', index=False)

print("Dados salvos no arquivo Excel com sucesso!")